In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
try: os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
except: print('Already does')
sys.path.insert(0, my_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import re, glob
import json
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors

In [3]:
display(glob.glob('drive/My Drive/kakao_arena/data/*'))
base_path = 'drive/My Drive/kakao_arena/'

['drive/My Drive/kakao_arena/data/val.json',
 'drive/My Drive/kakao_arena/data/test.json',
 'drive/My Drive/kakao_arena/data/genre_gn_all.json',
 'drive/My Drive/kakao_arena/data/train.json',
 'drive/My Drive/kakao_arena/data/song_meta.json',
 'drive/My Drive/kakao_arena/data/val_v1.json',
 'drive/My Drive/kakao_arena/data/test_v1.json',
 'drive/My Drive/kakao_arena/data/train_v1.json',
 'drive/My Drive/kakao_arena/data/genre_v1.json',
 'drive/My Drive/kakao_arena/data/song_meta_v1.json',
 'drive/My Drive/kakao_arena/data/test_v2.json',
 'drive/My Drive/kakao_arena/data/val_v2.json',
 'drive/My Drive/kakao_arena/data/train_v2.json',
 'drive/My Drive/kakao_arena/data/song_meta_v2.json']

In [4]:
train = pd.read_json(base_path + 'data/train_v2.json')
valid = pd.read_json(base_path + 'data/val_v2.json')
test = pd.read_json(base_path + 'data/test_v2.json')

In [5]:
# total = pd.concat([train.assign(group = 'train'),
#                    valid.assign(group = 'valid'),
#                    test.assign(group = 'test')], axis = 0).reset_index(drop = True)
# total.head(3)

In [6]:
song_meta = pd.read_json(base_path + 'data/song_meta_v2.json')
song_meta.head(3)

,song_gnr_dtl_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gnr_basket,artist_name_basket,songs,song_gnr_keywords,like_cnt,popular,song_name_basket,album_name_basket
0,[GN0901],20140512,불후의 명곡 7080 추억의 얄개시대 팝송베스트,2255639,[],feelings,[GN0900],[],0,"[국외, 팝]",0.058775,0.142951,[feeling],"[불후, 명곡, 추억, 시대, 팝송, 베스트]"
1,"[GN1601, GN1606]",20080421,bach partitas nos 2 3 4,376431,[29966],bach partita no 4 in d major bwv 828 ii allemande,[GN1600],[murray perahia],1,"[독주곡, 클래식]",0.000000,0.000000,"[bach, part, major, bwv, man]","[bach, part]"
2,[GN0901],20180518,hit,4698747,[3361],solsbury hill remastered 2002,[GN0900],[peter gabriel],2,"[국외, 팝]",0.000000,0.000000,[remaster],[hit]


In [7]:
normalization = lambda Series: (Series - np.min(Series)) / (np.max(Series) - np.min(Series))
song_meta['issue_date'] = normalization(song_meta.issue_date)
song_meta[['issue_date', 'like_cnt', 'popular']].head(3)

,issue_date,like_cnt,popular
0,0.925151,0.058775,0.142951
1,0.850077,0.000000,0.000000
2,0.975132,0.000000,0.000000


## 1. Song Embedding  

In [ ]:
album_id_list = song_meta.album_id.unique().tolist()
album_id_map = {e: i for i, e in enumerate(tqdm(album_id_list))}

artist_id_list = song_meta.artist_id_basket.explode().dropna().unique().tolist()
artist_id_map = {e: i + len(album_id_list) for i, e in enumerate(tqdm(artist_id_list))}

In [ ]:
def make_item(df):
    item_info = df.assign(
        album_id = lambda x: x.album_id.map(album_id_map).astype(str),
        artist_id_basket = lambda x: x.artist_id_basket.apply(lambda x: [str(artist_id_map[y]) for y in x]),
    )
    item_info = item_info.assign(
        gnr_art = lambda x: x.artist_id_basket + x.song_gnr_basket + x.song_gnr_dtl_basket,
        keywords = lambda x: x.song_gnr_keywords + x.song_name_basket + x.album_name_basket + x.artist_name_basket,
        others = lambda x: x.issue_date.apply(lambda x: [x]) + x.popular.apply(lambda x: [x]),
        total_items = lambda x: x.gnr_art + x.keywords
    )
    item_info['keywords'] = item_info.keywords.apply(lambda x: list(set(x)))
    return item_info[['gnr_art', 'keywords', 'others', 'total_items']]

In [ ]:
song_info = make_item(song_meta)
song_info.head(3)

,gnr_art,keywords,others,total_items
0,"[GN0900, GN0901]","[feeling, 팝, 추억, 불후, 국외, 베스트, 시대, 명곡, 팝송]","[0.9251508265004879, 0.1429508737]","[GN0900, GN0901, 국외, 팝, feeling, 불후, 명곡, 추억, 시..."
1,"[287235, GN1600, GN1601, GN1606]","[독주곡, bach, murray perahia, part, man, major, ...","[0.8500767719068543, 0.0]","[287235, GN1600, GN1601, GN1606, 독주곡, 클래식, bac..."
2,"[287236, GN0900, GN0901]","[remaster, 팝, hit, 국외, peter gabriel]","[0.9751318990083994, 0.0]","[287236, GN0900, GN0901, 국외, 팝, remaster, hit,..."


In [ ]:
def Embedding(Series, dim):
    items = Series.tolist()
    W2V = Word2Vec(items,
                   window = max([len(i) for i in items]), 
                   size = dim,
                   min_count = 3,
                   workers = 4,
                   negative = 5,
                   sg = 1)
    print('finish...')
    return W2V

def Aggregation(Series, main, sub):
    ID = []
    vec = []
    for idx in tqdm(Series.index):
        main_vec = 0
        main_N = 0
        for w in Series.at[idx, 'gnr_art']:
            try: main_vec += main.wv.get_vector(w)
            except: pass
            else: main_N += 1
        sub_vec = 0
        sub_N = 0
        for w in Series.at[idx, 'keywords']:
            try: sub_vec += sub.wv.get_vector(w)
            except: pass
            else: sub_N += 1
        main_vec /= main_N
        sub_vec /= sub_N
        total_vec = np.concatenate([main_vec, sub_vec, Series.at[idx, 'others']])

        ID.append(str(idx))
        vec.append(total_vec)
    vec = np.array(vec)
    
    WEKV = WordEmbeddingsKeyedVectors(vec.shape[1])
    WEKV.add(ID, vec)
    return WEKV

In [ ]:
main_embedding = Embedding(song_info.gnr_art, 127)

finish...


In [ ]:
sub_embedding = Embedding(song_info.keywords, 127)

finish...


In [ ]:
song2vec = Aggregation(song_info, main_embedding, sub_embedding)

In [ ]:
n = 144663
display(song_meta.iloc[[n]])
a = song2vec.most_similar_cosmul(str(n), topn = 100) # a[0]은 id, a[1]은 similarity가 나온다.
b = [int(x[0]) for x in a]
song_meta.loc[b]

,song_gnr_dtl_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gnr_basket,artist_name_basket,songs,song_gnr_keywords,like_cnt,popular,song_name_basket,album_name_basket
144663,"[GN0105, GN0101]",0.962396,밤편지,10047890,[261143],밤편지,[GN0100],[아이유],144663,"[발라드, 국내, 10년대]",0.946392,1.0,"[밤, 편지]","[밤, 편지]"


,song_gnr_dtl_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gnr_basket,artist_name_basket,songs,song_gnr_keywords,like_cnt,popular,song_name_basket,album_name_basket
70185,"[GN0105, GN0101]",0.963139,꽃갈피 둘,10096855,[261143],잠 못 드는 밤 비는 내리고,[GN0100],[아이유],70185,"[발라드, 국내, 10년대]",0.746297,0.830178,"[밤, 비]",[]
245317,"[GN0105, GN0101]",0.876048,real,1093562,[261143],첫 이별 그날 밤,[GN0100],[아이유],245317,"[발라드, 국내, 10년대]",0.864559,0.902173,"[이별, 밤]",[real]
674160,"[GN0105, GN0101]",0.988361,늦은 밤 너의 집 앞 골목길에서,10348811,[105623],늦은 밤 너의 집 앞 골목길에서,[GN0100],[노을],674160,"[발라드, 국내, 10년대]",0.614259,0.913580,[밤],[밤]
351342,"[GN0105, GN0101]",0.950646,웃긴 밤,2711116,[760046],끝,[GN0100],[권진아],351342,"[발라드, 국내, 10년대]",0.835701,0.922990,[],[밤]
468331,"[GN0105, GN0101]",0.975893,05,10226609,[420527],이 밤이 특별해진 건,[GN0100],[어반자카파],468331,"[발라드, 국내, 10년대]",0.628237,0.690709,[밤],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228798,"[GN0105, GN0101]",0.913163,여름 밤,2201331,"[474423, 181716]",여름 밤,[GN0100],"[슬옹, 에피톤 프로젝트]",228798,"[발라드, 국내, 10년대]",0.709142,0.653705,"[여름, 밤]","[여름, 밤]"
348172,"[GN0105, GN0101]",0.962391,스물다섯번째 밤,10047095,[858041],스물다섯번째 밤,[GN0100],[파인 (fine)],348172,"[발라드, 국내, 10년대]",0.089005,0.312013,[밤],[밤]
626428,"[GN0105, GN0101]",0.925651,가을에 만나,2281658,[14801],가을에 만나,[GN0100],[윤건],626428,"[발라드, 국내, 10년대]",0.600974,0.645763,[가을],[가을]
210647,"[GN0105, GN0101]",0.887302,real,1173161,[261143],나만 몰랐던 이야기,[GN0100],[아이유],210647,"[발라드, 국내, 10년대]",0.858835,0.908968,[이야기],[real]


In [ ]:
song2vec.save(base_path + 'models/song2vec_200719.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## Playlist Embedding

In [11]:
song_gnr_map = {idx: song_meta.at[idx, 'song_gnr_basket'] for idx in tqdm(song_meta.index)}
song_art_map = {idx: song_meta.at[idx, 'artist_name_basket'] for idx in tqdm(song_meta.index)}

In [12]:
def extend_items(df):
    song_ex = df.songs.explode().dropna()

    gnr_basket = song_ex.map(song_gnr_map)
    gnr_basket = gnr_basket.groupby(level = 0).agg(list)
    gnr_basket = gnr_basket.apply(lambda x: [z for y in x for z in y])
    gnr_basket = gnr_basket.apply(lambda x: list(set(x)))
    gnr_basket = gnr_basket.rename('gnr_basket')
    up_df = df.join(gnr_basket)

    art_basket = song_ex.map(song_art_map)
    art_basket = art_basket.groupby(level = 0).agg(list)
    art_basket = art_basket.apply(lambda x: [z for y in x for z in y])
    art_basket = art_basket.apply(lambda x: list(set(x)))
    art_basket = art_basket.rename('art_basket')
    up_df = up_df.join(art_basket)
    up_df = up_df.fillna('NAN').applymap(lambda x: [] if x == 'NAN' else x)
    return up_df

In [13]:
train = extend_items(train)
train.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,title_basket,tag_basket,gnr_basket,art_basket
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,20131219,"[여행, 음악]",[록],"[GN0900, GN1900, GN1400, GN1300, GN1000, GN1100]","[honeymoon suite, red hot chili peppers, onere..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,20141202,[요즘],"[추억, 회상]","[GN2600, GN0900, GN0100, GN0500, GN1600, GN170...","[manhattan jazz orchestra, 박지윤, little mix, ry..."
2,"[까페, 잔잔한]",76951,편하게 잔잔하게 들을 수 있는 곡,"[83116, 276692, 166267, 186301, 354465, 256598...",17,20170828,[잔잔],"[까페, 잔잔]","[GN2600, GN0100, GN0500, GN1700, GN0300, GN040...","[벤츄라, 참깨와 솜사탕, 타코앤제이형, common tale, 프롬유 (from...."


In [14]:
def make_items(df):
    user_info = df.assign(
        songs = lambda x: x.songs.apply(lambda x: list(map(str, x))),
        updt_date = lambda x: x.updt_date.apply(lambda x: str(x)[:4] + 'y')
    )
    user_info = user_info.assign(
        total_users = lambda x: x.tag_basket + x.songs + x.title_basket + x.gnr_basket + x.art_basket + x.updt_date.apply(lambda x: [x])
    )
    user_info['total_items'] = user_info.total_users.apply(lambda x: list(set(x)))
    return user_info.set_index('id')[['tags', 'songs', 'total_items']]

In [ ]:
train_items = make_items(train)
train_items.head(3)

In [20]:
valid = extend_items(valid)
valid.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,title_basket,tag_basket,gnr_basket,art_basket
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,20190527,[],[],"[GN0900, GN1300, GN1500, GN1000, GN2200]","[spencer lee, will smith, miley cyrus, agatha ..."
1,[],131447,앨리스테이블,[],1,20140716,[리스],[],[],[]
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,20080621,[],[],"[GN1500, GN0100, GN0400]","[성아, 아일리, 씨야, 윤화재인, 솔플라워 (sol` flower), 나영주, 가..."


In [21]:
valid_items = make_items(valid)
valid_items.head(3)

,tags,songs,total_items
id,,,
118598,[],"[373313, 151080, 275346, 696876, 165237, 52593...","[104455, spencer lee, 127698, will smith, 6161..."
131447,[],[],"[리스, 2014y]"
51464,[],"[529437, 516103, 360067, 705713, 226062, 37089...","[226062, 가비엔제이, 나영주, 린, 314668, h2가인, bmk, GN1..."


In [ ]:
%%time
user2vec = Embedding(plylst_info.total_items, 256)

In [ ]:
user2vec.save(base_path + 'models/user2vec_200719.model')

In [8]:
user2vec = Word2Vec.load(base_path + 'models/user2vec_200719.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [24]:
def Embedding_Mapper(df, model):
    ID = []
    vec = []
    for idx in tqdm(df.index):
        N = 0
        temp_vec = 0
        for w in df.at[idx, 'total_items']:
            try: temp_vec += model.wv.get_vector(w)
            except: pass
            else: N += 1
        if type(temp_vec) == int:
            continue
        ID.append(str(idx))
        vec.append(temp_vec / N)
    vec = np.array(vec)

    WEKV = WordEmbeddingsKeyedVectors(vec.shape[1])
    WEKV.add(ID, vec)
    return WEKV

In [25]:
train_mapped = Embedding_Mapper(train_items, user2vec)
valid_mapped = Embedding_Mapper(valid_items, user2vec)

In [30]:
n = 10532
display(train.query(f'id == {n}'))
a = train_mapped.most_similar_cosmul(str(n), topn = 100) # a[0]은 id, a[1]은 similarity가 나온다.
b = [int(x[0]) for x in a]
train.set_index('id').loc[b]

,tags,id,plylst_title,songs,like_cnt,updt_date,title_basket,tag_basket,gnr_basket,art_basket
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,20141202,[요즘],"[추억, 회상]","[GN2600, GN0900, GN0100, GN0500, GN1600, GN170...","[manhattan jazz orchestra, 박지윤, little mix, ry..."


,tags,plylst_title,songs,like_cnt,updt_date,title_basket,tag_basket,gnr_basket,art_basket
id,,,,,,,,,
26290,[발라드],나 오늘밤에야,"[315216, 192882, 383960, 548636, 414721, 75801...",0,20141125,"[오늘, 밤]",[발라드],"[GN2600, GN0900, GN0100, GN0500, GN1700, GN150...","[above envy, 이승환, chihiro yamazaki+route 14 ba..."
3553,"[휴식, 힐링]",널 어쩌면 좋을까,"[675945, 120377, 389529, 50833, 129814, 61435,...",1,20141208,[],"[휴식, 힐링]","[GN2600, GN0900, GN0100, GN1900, GN0500, GN170...","[이승환, above envy, 서태지, 이승철, 박명수, lim kim, 라이너스..."
153270,[겨울],호빵호빵,"[432406, 675945, 120377, 389529, 315216, 19288...",0,20141201,[],[겨울],"[GN2600, GN0900, GN0100, GN0500, GN1700, GN150...","[above envy, 이승환, chihiro yamazaki+route 14 ba..."
11853,[기분전환],자꾸 너라서,"[175244, 641118, 400359, 382473, 25029, 453073...",0,20141209,[],[기분전환],"[GN2600, GN0900, GN1900, GN0100, GN0500, GN140...","[박지윤, louis yoelin, little mix, ryu miho, 이상순,..."
114918,[재즈],미끌미끌,"[55670, 225176, 442765, 244000, 692078, 645653...",1,20141203,[],[재즈],"[GN0100, GN0500, GN1700, GN1500, GN1000, GN060...","[above envy, 이승환, chihiro yamazaki+route 14 ba..."
...,...,...,...,...,...,...,...,...,...
7921,"[피크닉, 밖으로, 음악모음, 싱그러운, 감성노래, 나들이, 산책]",한강공원에 돗자리 깔고 누워보자,"[215398, 501825, 704882, 139490, 175244, 80213...",4,20170423,[한강],"[음악, 모음, 감성, 노래, 산책]","[GN0900, GN0100, GN0500, GN1400, GN1700, GN150...","[정재일, fourplay, above envy, chihiro yamazaki+r..."
76604,"[힐링, 휴식, 기분전환]",상 주고싶은 hot 아티스트들 모아봤다,"[161970, 454075, 223580, 41432, 635230, 148908...",2,20150211,"[hot, 아티스트]","[힐링, 휴식, 기분전환]","[GN2600, GN0900, GN0500, GN1600, GN1700, GN140...","[fortune, chihiro yamazaki+route 14 band, 서태지,..."
61872,"[가을, 까페]",카페에 앉아 널 기다려,"[39550, 579678, 461062, 610933, 512750, 2955, ...",7,20151022,[카페],"[가을, 까페]","[GN0900, GN0100, GN1900, GN0500, GN1700, GN130...","[komoguchi yuya, keira knightley, kazumi tatei..."


In [35]:
cold_start = pd.read_json(base_path + 'submissions/ColdStart_2.json')
cold_start.head(3)

,id,songs,tags
0,118598,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
1,131447,"[326204, 695032, 387859, 112399, 5049, 318487,...","[락, 겨울, 크리스마스, 기분전환, 캐롤, 명곡, 팝, 블루스, 클래식, Pop]"
2,51464,"[654757, 253755, 117595, 645489, 13198, 88503,...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"


In [56]:
tag_dic = {str(train.at[idx, 'id']): train.at[idx, 'tags'] for idx in tqdm(train.index)}
song_dic = {str(train.at[idx, 'id']): train.at[idx, 'songs'] for idx in tqdm(train.index)}
date_dic = {idx: song_meta.at[idx, 'issue_date'] for idx in tqdm(song_meta.index)}

In [36]:
def remove_seen(seen, items):
    seen = set(seen)
    return [i for i in items if i not in seen]

In [57]:
def rec(df, base_df):
    # recommendation
    res = []
    for idx in tqdm(df.index):
        songs, tags, date, ID = df.at[idx, 'songs'], df.at[idx, 'tags'], df.at[idx, 'updt_date'], df.at[idx, 'id']
        try:
            most_id = [x[0] for x in train_mapped.similar_by_vector(valid_mapped.get_vector(str(ID)), topn = 200)]
            get_songs, get_tags = [], []
            for i in most_id:
                get_songs += song_dic[i]
                get_tags += tag_dic[i]
            get_songs = list(pd.value_counts(get_songs)[: 200].index)
            get_tags = list(pd.value_counts(get_tags)[: 20].index)

            get_songs = remove_seen(songs, get_songs)[:100]
            get_tags = remove_seen(tags, get_tags)[: 10]

            if len(get_songs) < 100:
                get_songs += remove_seen(get_songs, base_df.at[idx, 'songs'])[: 100 - len(get_songs)]
            if len(get_tags) < 10:
                get_tags += remove_seen(get_tags, base_df.at[idx, 'tags'])[: 10 - len(get_tags)]

            res.append({
                'id': ID,
                'songs': get_songs,
                'tags': get_tags
            })
        except:
            res.append({
                'id': ID,
                'songs': base_df.at[idx, 'songs'],
                'tags': base_df.at[idx, 'tags']
            })
    res = pd.DataFrame(res)
    return res

In [58]:
result = rec(valid, cold_start)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [60]:
result[result.songs.apply(len) != 100]

,id,songs,tags


In [61]:
result.to_json(base_path + 'submissions/Item2Vec_200720_1.json', orient = 'records', force_ascii = False)